In [ ]:
!pip install mediapipe

In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
from PIL import ImageFont, ImageDraw, Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.decomposition import PCA
from keras import models

In [ ]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join(r"/Users/krama/Desktop/Sign Language/Sign Language Dataset/Data") 

subfolders = [ f.name for f in os.scandir(DATA_PATH) if f.is_dir() ]
print(subfolders)

# Actions that we try to detect
actions = subfolders
# Thirty videos worth of data
no_sequences = 100


# Videos are going to be 60 frames in length
sequence_length = 60




In [ ]:
label_map = {label:num for num, label in enumerate(actions)}
label_map

In [ ]:
sequences,labels=[],[]
for action in actions:
    for sequence in range(no_sequences):
        window=[]
        for frame_num in range(sequence_length):
            
            res=np.load(os.path.join(DATA_PATH, action, str(sequence), str(frame_num) + ".npy"))
            pose=res[:33*4]#pose
            hands=res[33*4:]
            
            pose=pose.reshape(-1,4)
            hands=hands.reshape(-1,3)
            
            pose1=pose[:,0:3]-pose[0,0:3]
            hand1=hands[:,:]-pose[0,0:3]
            hand1[hands==0]=0
            pose1=np.concatenate([np.expand_dims(pose1[0],axis=0),pose1[11:23]],axis=0).flatten()
            all_pt=np.concatenate([pose1,hand1.flatten()],axis=0)
            window.append(all_pt)
        sequences.append(window)
        labels.append(label_map[action])

In [ ]:
np.array(sequences).shape 

In [ ]:
np.save(r"/Users/krama/Desktop/Sign Language/Sign Language Dataset/Refined_New_X_100",np.array(sequences))

In [ ]:
np.array(labels).shape

In [ ]:
np.save(r"/Users/krama/Desktop/Sign Language/Sign Language Dataset/Refined_New_Y_100",np.array(labels))

In [ ]:
# colab
# sequences = np.load('/content/drive/MyDrive/AI Python/Dataset/Refined_New_X.npy')
# labels = np.load('/content/drive/MyDrive/AI Python/Dataset/Refined_New_Y.npy')

In [ ]:
X = np.array(sequences)

In [ ]:
X.shape

In [ ]:
y = to_categorical(labels).astype(int)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [ ]:
y_test.shape

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, GRU
from tensorflow.keras.callbacks import TensorBoard


model = Sequential()
model.add(GRU(512, return_sequences=True, activation='elu', input_shape=(60,165)))
model.add(GRU(1024, return_sequences=True, activation='elu'))
model.add(GRU(512, return_sequences=False, activation='elu'))
model.add(Dense(512, activation='elu'))
model.add(Dense(256, activation='elu'))
model.add(Dense(np.array(actions).shape[0], activation='softmax'))
model.summary()


In [ ]:
from tensorflow.keras.optimizers import Adam
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['categorical_accuracy'])
import keras
from keras.callbacks import ModelCheckpoint, EarlyStopping
#checkpoint = keras.callbacks.ModelCheckpoint('/content/drive/MyDrive/AI Python/Dataset/Dropout.h5', monitor='val_categorical_accuracy', mode="max", save_best_only=True, verbose=1)
earlystopping = keras.callbacks.EarlyStopping(monitor='val_loss', mode="auto", patience=3, verbose=1)
rlr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1, verbose=1, mode='auto')


model.fit(X_train, y_train,validation_split = 0.1,
              epochs=30,
              batch_size = 32,
              callbacks=[rlr,earlystopping])

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'test_loss={test_loss}')
print(f'test_accuracy={test_accuracy}')

In [ ]:
model.save('action.h5')